The damped least squares method is an improvment to the pseudoinverse method. This algorithm uses the least squares method similar to the pseudoinverse method but also uses a damping factor to avoid singularities. The damping factor is a metavariable that needs to be tuned based on the configuration space.

In [1]:
import numpy as np
import sympy as sym
import math

In [2]:
# SOME HARD CODED VALUES FOR ROBOT PARAMETERS
z = 0.3
l1 = 0.5
l2 = 0.5
r1 = 0.2
r2 = 0.25
r3 = 0.3
phi_value = 0
theta1_value = 1.5714
theta2_value = 0
theta3_value = 1.5714
starting_angles = sym.Matrix([phi_value, theta1_value, theta2_value, theta3_value])
phi, theta1, theta2, theta3 = sym.symbols("phi theta1 theta2 theta3")


In [3]:
# MAKE A DH PARAMETER TABLE
# d, alpha, r, theta
dh_parameters = sym.Matrix(
  [
   [0, 0, 0, 0],
   [z + r1, 1.57, 0, phi],
   [0, 0, r1 + l1 + r2, theta1],
   [0, 0, r2 + l2 + r3, theta2],
   [0, 0, r3, theta3]
  ] 
)
print(dh_parameters)


Matrix([[0, 0, 0, 0], [0.500000000000000, 1.57000000000000, 0, phi], [0, 0, 0.950000000000000, theta1], [0, 0, 1.05000000000000, theta2], [0, 0, 0.300000000000000, theta3]])


In [4]:

# MAKE A DH MATRIX FOR THE ROBOT

# USING THE PARAMETERS 
dh_matrix = np.diag(np.ones((4)))

print(dh_parameters.shape[0])
for i in range(dh_parameters.shape[0]):
  
  trans_z = sym.Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, dh_parameters[i, 0]],
    [0, 0, 0, 1]
  ])
  dh_matrix = dh_matrix * trans_z
  rot_z = sym.Matrix([
    [sym.cos(dh_parameters[i, 3]), -1 * sym.sin(dh_parameters[i, 3]), 0, 0],
    [sym.sin(dh_parameters[i, 3]), sym.cos(dh_parameters[i, 3]), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0 ,1]
  ])
  dh_matrix = dh_matrix * rot_z
  trans_x = sym.Matrix([
    [1, 0, 0, dh_parameters[i, 2]],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
  ])
  dh_matrix = dh_matrix * trans_x 
  rot_x = sym.Matrix([
    [1, 0, 0, 0],
    [0, sym.cos(dh_parameters[i, 1]), -1 * sym.sin(dh_parameters[i, 1]), 0],
    [0, sym.sin(dh_parameters[i, 1]), sym.cos(dh_parameters[i, 1]), 0],
    [0, 0, 0, 1]
  ])
  dh_matrix =  dh_matrix * rot_x
  
  
print(dh_matrix) 

5
Matrix([[(-(-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*sin(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*cos(theta2))*sin(theta3) + ((-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*cos(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*sin(theta2))*cos(theta3), (-(-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*sin(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*cos(theta2))*cos(theta3) - ((-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*cos(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*sin(theta2))*sin(theta3), 0.999999682931835*sin(phi), 0.3*(-(-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*sin(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*cos(theta2))*sin(theta3) + 0.3*((-0.00079632671073326

In [5]:
# GET START POSITION
position_function = dh_matrix[0:3, -1]
start_position = position_function.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
current_position = start_position
print(start_position)

Matrix([[-0.301207127684100], [0.00159236469867950], [2.49963679772224]])


In [6]:
# SET END POSITION

target_position = sym.Matrix([1, 1, 1])


In [7]:
# ALGORITHM

# CALCULATE THE JACOBIAN
angles = sym.Matrix([phi, theta1, theta2, theta3])
# 
jacobian_matrix = position_function.jacobian(angles)


In [8]:
# #INPUT DAMPING FACTOR 
# print('Enter Damping Factor')
# damping_factor = float(input())

In [11]:
starting_phi_value = phi_value
starting_theta1_value = theta1_value
starting_theta2_value = theta2_value
starting_theta3_value = theta3_value

for damping_factor in range(0, 10):
  phi_value = starting_phi_value
  theta1_value = starting_theta1_value
  theta2_value = starting_theta2_value
  theta3_value = starting_theta3_value
  for i in range(10):
    
    jacobian_matrix_values = jacobian_matrix.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
    # CALCULATE THE JACOBIAN TRANSPOSE
    jacobian_transpose = jacobian_matrix_values.transpose()
    difference_position = target_position - position_function
    difference_position_value = difference_position.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
    
    jjt = jacobian_matrix_values * jacobian_transpose

    identity_matrix = sym.Matrix.eye(jjt.shape[0])
    inverse_matrix = (jjt + (damping_factor / 10) ** 2 * identity_matrix).inv()
    delta_theta = jacobian_transpose * inverse_matrix * difference_position_value
    delta_theta_value = delta_theta.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
      
    phi_value += delta_theta_value[0]
    theta1_value += delta_theta_value[1]
    theta2_value += delta_theta_value[2]
    theta3_value += delta_theta_value[3]
    
    current_position = position_function.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
    print(current_position)

Matrix([[0.332714707096219], [0.101202811478566], [1.16492493634243]])
Matrix([[-0.613782275675482], [0.821772232660095], [0.476071461171401]])
Matrix([[0.176012659223661], [0.199230598393695], [0.433890852596259]])
Matrix([[0.589364819691513], [0.335265714235513], [1.25743539548882]])
Matrix([[0.600654867664229], [1.09832686884816], [0.729447522621486]])
Matrix([[0.964121129939635], [0.903198113089285], [1.01878968335197]])
Matrix([[0.996220520349627], [1.00080027618800], [0.996866104744504]])
Matrix([[0.999996766625556], [0.999989941941346], [1.00000291586944]])
Matrix([[0.999999999959496], [1.00000000000485], [0.999999999965133]])
Matrix([[1.00000000000000], [0.999999999999998], [1.00000000000000]])
Matrix([[-0.358993130119862], [-0.0519412106251744], [1.12839994370078]])
Matrix([[-0.0259158205522622], [-0.0523236481288838], [0.443773976968192]])
Matrix([[-1.06357925653824], [0.0697894926009654], [-0.255088351496849]])
Matrix([[-0.0583726568412888], [0.100352563471643], [1.930312088